# Лабораторная работа №5 Численное интегрирование

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
epsilon = 0.00001

def is_zero (num):
    if(np.abs(num) < epsilon):
        return 0.0
    else:
        return num
        
def print_matrix(name, A, n, m):
    print("\n", "Matrix", name, "(", n, "x", m, "):")
    
    for i in range(n):
        print("||", end = ' ')
        for j in range(m):
            if(is_zero(A[i][j]) >= 0):
                print(" ", end = '')
            print(format(is_zero(A[i][j]), '5.5f'), end = ' ')
        print("||")
    
    print("\n")
    
    return

def vec_norm (v, n):
    res = 0.0
    for i in range(n):
        res = res + v[i] ** 2
    res = res ** (0.5)
    return res

def matrix_dot (A, x, n):
    res = np.zeros((n, 1))
    for i in range(n):
        for j in range(n):
            res[i] += A[i][j] * x[j]

    return res

def skalar_dot (a, b, n):
    res = 0.0
    for i in range(n):
        res += a[i] * b[i]

    return res


## 1) Формулы Ньютона-Котеса 
Вычислим интеграл от поточечно заданной функции из $VII.9.5(a)$ при помощи методов трапеций и Симпсона.\
\
a) Метод трапеций (основан на аппроксимации функции прямой между шагами):
$ I \approx \sum_{k = 0}^{n - 1} \frac{h_k}{2}\left(f(x_{k + 1}) + f(x_k))\right), \quad h_k = x_{k + 1} - x_k $\
Уточним его значение при помощи правила Рунге:
$ I \approx I^p(h) + \frac{I^p(h) - I^p(2h)}{2^p - 1} $\
При этом должно выполняться:
$ \left|2^p \cdot \frac{I^p(h) - I^p(2h)}{I^p(2h) - I^p(4h)} - 1 \right| < 0.1 $ (enable)\
В нашем случае $h = 0.25$.\
\
b) Метод Симпсона (аппроксимируем три точки параболой и от нее берем интеграл на каждом интервале):\
$  I \approx \sum_{k = 0}^{[n/2] } \frac{h_2k}{3}\left(f(x_{2k}) + 4 \cdot f(x_{2k + 1}) + f(x_{2k + 2}))\right) $

In [3]:
N   = 9
h   = 0.25 
h2  = h * 2
h4  = h * 4

X  = np.zeros(N, dtype = np.float64)
Y = np.array([1.0, 0.989616, 0.958851, 0.908852, 0.841471, 0.759188, 0.664997, 0.562278, 0.454649])

for i in range(1, N):
    X[i] = X[i - 1] + h

#---------- Trapezoid method ---------#
I_trap_h  = 0.0
I_trap_h2 = 0.0
I_trap_h4 = 0.0

for i in range(N - 1):
    I_trap_h += ((Y[i + 1] + Y[i]) * h / 2)

for i in range(0, N - 1, 2):
    I_trap_h2 += ((Y[i + 1] + Y[i]) * h2 / 2)

for i in range(0, N - 1, 4):
    I_trap_h4 += ((Y[i + 1] + Y[i]) * h4 / 2)


delta_I = (I_trap_h - I_trap_h2) / (2**2 - 1)
enable = np.abs(2**2 * (I_trap_h - I_trap_h2) / (I_trap_h2 - I_trap_h4) - 1)

#---------- Simpson method ----------#
I_simp = 0.0

for i in range((N - 1) // 2):
    I_simp += (Y[2 * i] + 4 * Y[2 * i + 1] + Y[2 * i + 2]) * (h / 3)

print("Enable: ", enable)
print("A result of the trapezoid method: I = ", format(I_trap_h, '5.5f'), ", I + dI = ", format(I_trap_h + delta_I, '5.5f'))
print("A result of the Simpson method: I = ", format(I_simp, '5.5f'))

Enable:  1.20211711357025
A result of the trapezoid method: I =  1.60314 , I + dI =  1.58042
A result of the Simpson method: I =  1.60542


Видим, что метод Симпсона дал более точный результат (точным считается 1.60541, полученный в Desmos аппроксимацией полиномом 4-ой степени). Правило Рунге в нашем случае для метода трапеций применить нельзя, но несмотря на это результат близок к истине (к сожалению, функция задана небольшим количеством точек).

## 2) Интегрирование быстро осциллирующей функции
Решим задачу $VII.9.11(a)$. Такой интеграл можно заменить следующей суммой:\
$ I \approx \sum_{i = 0}^{N} c_i \cdot f(x_i) $\
$c_i, x_i$ находим из системы:\
$ \sum_{i = 0}^{N} c_i \cdot x_i^k = \int_{a}^{b} x^k \cdot sin(50x) dx, \qquad  k = 0, 1.., N $

In [4]:
I = np.array([0.010256, -0.039324, -0.15870, -0.64027])


def f_1(X):
    return X[0] + X[1] - I[0]
    
def f_2(X):
    return X[0] * X[2] + X[1] * X[3] - I[1]

def f_3(X):
    return X[0] * (X[2]**2) + X[1] * (X[3]**2) - I[2]

def f_4(X):
    return X[0] * (X[2]**3) + X[1] * (X[3]**3) - I[3]

funcs = [f_1, f_2, f_3, f_4]
    
def F(X):
    res = np.zeros((4, 1))

    for i in range(4):
        res[i] = funcs[i](X)

    return res

def G(X):
    res        = np.zeros((4, 4))
    X_dx_plus  = np.zeros((4, 1))
    X_dx_minus = np.zeros((4, 1))
    
    h = 2**(-4)

    for i in range(4):
        for j in range(4):
            for k in range(4):
                if(k == j):
                    X_dx_plus [k] = X[k] + h
                    X_dx_minus[k] = X[k] - h
                else:
                    X_dx_plus [k] = X[k]
                    X_dx_minus[k] = X[k]
                    
            res[i][j] = (funcs[i](X_dx_plus) - funcs[i](X_dx_minus)) / (2 * h)
    return res

In [5]:

#---------- Newton method ----------#
x_curr = np.zeros((4, 1))
x_next = np.zeros((4, 1))
delt_x = np.zeros((4, 1))
A      = np.zeros((4, 4))
b      = np.zeros((4, 1))

x_curr[0] = 1
x_curr[1] = 1
x_curr[2] = 4
x_curr[3] = 0

count_new = 0
count_ar_new = np.zeros(40)
x_res_new = np.zeros(40)

for i in range(40):
    count_ar_new[i] = i + 1



while is_zero(vec_norm(F(x_curr), 4)) != 0 and count_new < 40:
    A = G(x_curr)
    b = - F(x_curr)
    #---------- Gauss method for system "A delta_x = b" ----------#
    n = 4
    x_inter = np.zeros(n)
    matrix  = np.zeros((n, n + 1))

    for i in range(n):
        for j in range(n):
            matrix[i][j] = A[i][j]
        matrix[i][n] = b[i]

    for i in range(n):
        max_row = i + np.argmax(np.abs(matrix[i:, i]))
        matrix[[i, max_row]] = matrix[[max_row, i]]

        for j in range(i + 1, n):
            factor = matrix[j][i] / matrix[i][i]
            matrix[j] = matrix[j] - factor * matrix[i]

    for i in range(n - 1, -1, -1):
        x_inter[i] = (matrix[i][-1] - np.dot(matrix[i][i + 1:n], x_inter[i + 1:n])) / matrix[i][i]

    for i in range(n):
        delt_x[i] = x_inter[i]
    #--------------------------------------------------------#

    x_next = delt_x + x_curr
    x_curr = x_next
    x_res_new[count_new] = is_zero(vec_norm(F(x_curr), 4))
    count_new += 1

x_new = x_curr

print_matrix("x_new", x_new, 4, 1)
print("Cnt_new:", count_new)


 Matrix x_new ( 4 x 1 ):
|| -0.00975 ||
||  0.02001 ||
||  4.03447 ||
||  0.00060 ||


Cnt_new: 3


Выше мы нашли столбец $\vec{x_{new}} = (c_0, c_1, x_0, x_1)^T $. Теперь осталось аппроксимировать функцию $f(x)$, которая задана точками, для получения значений в нужных $x$. Сделаем это вначале, как требуется в задании, при помощи естественного кубического сплайна. 

In [6]:
#----------Spline approximation----------#
def natural_cubic_spline(x, y):
    n = len(x) - 1
    h = np.diff(x)
    alpha = np.zeros(n)
    
    # Evalution of alpha
    for i in range(1, n):
        alpha[i] = (3 / h[i]) * (y[i + 1] - y[i]) - (3 / h[i - 1]) * (y[i] - y[i - 1])
    
    # Matrix for system of equations
    l = np.ones(n + 1)
    mu = np.zeros(n)
    z = np.zeros(n + 1)
    
    for i in range(1, n):
        l[i] = 2 * (x[i + 1] - x[i - 1]) - h[i - 1] * mu[i - 1]
        mu[i] = h[i] / l[i]
        z[i] = (alpha[i] - h[i - 1] * z[i - 1]) / l[i]
    
    l[n] = 1
    z[n] = 0
    c = np.zeros(n + 1)
    
    # Running method for c
    for j in range(n - 1, -1, -1):
        c[j] = z[j] - mu[j] * c[j + 1]
    
    b = np.zeros(n)
    d = np.zeros(n)
    
    for i in range(n):
        b[i] = (y[i + 1] - y[i]) / h[i] - h[i] * (c[i + 1] + 2 * c[i]) / 3
        d[i] = (c[i + 1] - c[i]) / (3 * h[i])
    
    return b, c[:-1], d

def Spline_pol(x, y, b, c, d, x_eval):
    n = len(x) - 1
    y_eval = np.zeros_like(x_eval)
    
    for i in range(n):
        mask = (x_eval >= x[i]) & (x_eval <= x[i + 1])
        if np.any(mask):
            dx = x_eval[mask] - x[i]
            y_eval[mask] = y[i] + b[i] * dx + c[i] * dx**2 + d[i] * dx**3

    for i in range(len(x_eval)):
        if x_eval[i] > x[n]:
            dx = x_eval[i] - x[n]
            y_eval[i] = y[n - 1] + b[n - 1] * dx + c[n - 1] * dx**2 + d[n - 1] * dx**3

        elif x_eval[i] < x[0]:
            dx = x[0] - x_eval[i]
            y_eval[i] = y[0] + b[0] * dx + c[0] * dx**2 + d[0] * dx**3
            
            
        
            
    return y_eval

x = np.array([0.0, 1.0, 2.0, 3.0, 4.0])
y = np.array([0.00000, 0.50000, 0.86603, 1.00000, 0.86603])

b, c, d = natural_cubic_spline(x, y)

x_eval = np.array([x_new[2], x_new[3]])
y_eval = Spline_pol(x, y, b, c, d, x_eval)

I_res = x_new[0] * y_eval[0] + x_new[1] * y_eval[1]

print("A result for spline approximation: I = ", I_res)


A result for spline approximation: I =  [-0.00973539]


Также воспользуемся формой Ньютона.

In [7]:
n = 5

#----------Newton approximation----------#
def Newton_pol (x_eval, x, coeff):
    res = 0.0;
    mul = 1.0;

    for i in range(n):
        for j in range(0, i):
            mul *= (x_eval - x[j])

        res += coeff[i] * mul
        mul = 1.0;

    return res

temp_coeff = np.zeros((n, n))
coeff = np.zeros(n, dtype = np.float64)

for i in range(n):
    temp_coeff[0][i] = y[i]

for i in range(1, n):
    for j in range(0, n - i):
        temp_coeff[i][j] = (temp_coeff[i - 1][j + 1] - temp_coeff[i - 1][j]) / (x[j + i] - x[j])

for j in range(n):
    coeff[j] = temp_coeff[j][0]

y_eval_new = np.zeros((2, 1))
y_eval_new[0] = Newton_pol(x_eval[0], x, coeff)
y_eval_new[1] = Newton_pol(x_eval[1], x, coeff)

I_res_new = x_new[0] * y_eval_new[0] + x_new[1] * y_eval_new[1]

print("A result for Newton approximation: I = ", I_res_new)


A result for Newton approximation: I =  [-0.00834728]


# Выводы
В данной лабораторной работе мы ознакомились с базовыми алгоритмами численного интегрирования - методами Ньютона-Контеса, а также разобрали более сложный пример с быстро осциллирующей функцией, где первые методы нерационально применять.

Было проведено сравнение методов трапеций и Симпсона. Очевидно, что при равных разбиениях второй дал более точный результат. К сожалению, провести уточнение результата метода трапеций при помощи правила Рунге не удалось, так как нарушалось необходимое условие применимости.

Наконец, разобрались с быстро осциллирующей функцией. Здесь после получения формулы для вычисления интеграла по весовым значениям функции получились два довольно разнящихся результата (сплайн-интерполяция дала больший). Заметим, что за ориентировочное значение бралось -0.00834716, полученное аппроксимацией функции в Desmos и дальнейшем вычислении интеграла с синусом. Результат в случае со сплайном сильно отличается, поскольку надо было брать одну точку функции за пределом отрезка интегрирования (4.03447) и определения функциии, что подразумевает экстраполяцию.